In [1]:
import pandas as pd
import dask.dataframe as dd
from BayDS.lib.utils import *
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from dask.diagnostics import ProgressBar
ProgressBar().register()
from tqdm import tqdm
tqdm.pandas()
import multiprocess as mp
from BayDS.lib.aggregations.temporal import *

In [39]:
# best_features = dd.read_parquet('./FE/best_features_1141.pqt')
data = pd.read_pickle('./FE/initial_data.pkl')

In [40]:
num_features = ['field0', 'price1', 'field6', 'field9', 'field11', 
               'field13', 'price2', 'field15', 'field16', 'field17', 
              'field19', 'field22', 'field23', 'field24', 'field25', 'field28' ]
data_slice = data[['field4','userid'] +  num_features].reset_index()

In [41]:
# data_slice = data[[date_field, group_by_feature, nf]].reset_index()
window = [2, 3, 4, 5, 10, 20]
args = []
for nf in num_features:
    args += [(data_slice, 'userid', nf, ws, 'field4', data.index.name, True) for ws in window]
    args += [(data_slice, 'userid', nf, ws, 'field4', data.index.name, False) for ws in window]

In [42]:
len(args)

192

In [18]:
df = pd.DataFrame(index=data_slice.index)
with mp.Pool(processes=4) as Pool:
    m = Pool.imap(aggregate_with_time_local, args)

    for i, df_agg in enumerate(m):
        print('.')
        assert data_slice.shape[0] == df_agg.shape[0]
        df = pd.concat([df, df_agg], axis=1)

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


In [19]:
# df.to_pickle('./FE/sequential.pkl')

In [2]:
df = pd.read_pickle('./FE/sequential.pkl')

In [3]:
def missing_values_table(df):
    '''
    show count and percent of missing values in dataframe
    :param df: dataframe
    :return: stats of missings
    '''
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: 'Missing Values', 1: '% of Total Values'})

    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

    print("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
                                                              "There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns that have missing values.")
    return mis_val_table_ren_columns

In [34]:
missing_values_table(df_corrected)

Your selected dataframe has 384 columns.
There are 372 columns that have missing values.


,Missing Values,% of Total Values
field9_by_userid_ws2_to_std,641478,98.5
field9_by_userid_ws2_to_std,641478,98.5
field9_by_userid_ws3_to_std,639808,98.3
field9_by_userid_ws4_to_std,639105,98.2
field9_by_userid_ws5_to_std,638138,98.0
...,...,...
field24_by_userid_ws20_to_mean,1,0.0
field24_by_userid_ws3_to_mean,1,0.0
field24_by_userid_ws4_to_mean,1,0.0
field24_by_userid_ws5_to_mean,1,0.0


In [28]:
df_corrected = df.drop(list(set(data_slice.index) - set(data.index) ))
df_corrected.index.name = 'orderid'

In [32]:
df_corrected = df_corrected.loc[data.index]

In [33]:
df_corrected.to_pickle('./FE/sequential_corrected.pkl')

In [64]:
df_corrected = pd.read_pickle('./FE/sequential_corrected.pkl')

In [44]:
_, i = np.unique(df_corrected.columns, return_index=True)

In [66]:
df_renamed=pd.DataFrame(index=df_corrected.index)

already=set()
for i,col in enumerate(df_corrected.columns):
    if col not in already:
        print(col)
        df_renamed[col] = df_corrected.iloc[:,i]
        already.add(col)
    else:
        print(col+'_right')
        df_renamed[col+'_right'] = df_corrected.iloc[:,i]

field0_by_userid_ws2_to_mean
field0_by_userid_ws2_to_std
field0_by_userid_ws3_to_mean
field0_by_userid_ws3_to_std
field0_by_userid_ws4_to_mean
field0_by_userid_ws4_to_std
field0_by_userid_ws5_to_mean
field0_by_userid_ws5_to_std
field0_by_userid_ws10_to_mean
field0_by_userid_ws10_to_std
field0_by_userid_ws20_to_mean
field0_by_userid_ws20_to_std
field0_by_userid_ws2_to_mean_right
field0_by_userid_ws2_to_std_right
field0_by_userid_ws3_to_mean_right
field0_by_userid_ws3_to_std_right
field0_by_userid_ws4_to_mean_right
field0_by_userid_ws4_to_std_right
field0_by_userid_ws5_to_mean_right
field0_by_userid_ws5_to_std_right
field0_by_userid_ws10_to_mean_right
field0_by_userid_ws10_to_std_right
field0_by_userid_ws20_to_mean_right
field0_by_userid_ws20_to_std_right
price1_by_userid_ws2_to_mean
price1_by_userid_ws2_to_std
price1_by_userid_ws3_to_mean
price1_by_userid_ws3_to_std
price1_by_userid_ws4_to_mean
price1_by_userid_ws4_to_std
price1_by_userid_ws5_to_mean
price1_by_userid_ws5_to_std
price1_b

field22_by_userid_ws3_to_std_right
field22_by_userid_ws4_to_mean_right
field22_by_userid_ws4_to_std_right
field22_by_userid_ws5_to_mean_right
field22_by_userid_ws5_to_std_right
field22_by_userid_ws10_to_mean_right
field22_by_userid_ws10_to_std_right
field22_by_userid_ws20_to_mean_right
field22_by_userid_ws20_to_std_right
field23_by_userid_ws2_to_mean
field23_by_userid_ws2_to_std
field23_by_userid_ws3_to_mean
field23_by_userid_ws3_to_std
field23_by_userid_ws4_to_mean
field23_by_userid_ws4_to_std
field23_by_userid_ws5_to_mean
field23_by_userid_ws5_to_std
field23_by_userid_ws10_to_mean
field23_by_userid_ws10_to_std
field23_by_userid_ws20_to_mean
field23_by_userid_ws20_to_std
field23_by_userid_ws2_to_mean_right
field23_by_userid_ws2_to_std_right
field23_by_userid_ws3_to_mean_right
field23_by_userid_ws3_to_std_right
field23_by_userid_ws4_to_mean_right
field23_by_userid_ws4_to_std_right
field23_by_userid_ws5_to_mean_right
field23_by_userid_ws5_to_std_right
field23_by_userid_ws10_to_mean_righ

In [76]:
df_renamed.to_pickle('./FE/sequential_corrected_rename.pkl')